In [21]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

batch = torch.stack((inputs,inputs),dim=0)
print(batch.shape)

torch.Size([2, 6, 3])


In [28]:
class CausalAttention(torch.nn.Module):
    def __init__(self,d_in,d_out,context_length,dropout,qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = torch.nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_key = torch.nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_value = torch.nn.Linear(d_in,d_out,bias=qkv_bias)
        self.dropout = torch.nn.Dropout(dropout)
        self.register_buffer('mask',torch.triu(torch.ones(context_length,context_length),diagonal=1))

    def forward(self,x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attention_scores = queries @ keys.transpose(1,2)
        attention_scores.masked_fill_(
            self.mask.bool()[:num_tokens,:num_tokens],-torch.inf)
        attention_weights = torch.softmax(
            attention_scores / keys.shape[-1] ** 0.5,dim=-1
        )
        attention_weights = self.dropout(attention_weights)

        context_vector = attention_weights @ values
        return context_vector

In [29]:
class MultiHeadAttention(torch.nn.Module):
    def __init__(self,d_in,d_out,context_length,dropout,num_heads,qkv_bias=False):
        super().__init__()
        self.heads = torch.nn.ModuleList(
            [CausalAttention(d_in,d_out,context_length,dropout,qkv_bias) 
             for _ in range(num_heads)]
        )
    
    def forward(self,x):
        return torch.cat([head(x) for head in self.heads],dim=-1) ## each head's
        ## output is concatenated along the last dimension
    


In [30]:
torch.manual_seed(123)
context_length = batch.shape[1]

d_in,d_out = 3,2

mha = MultiHeadAttention(d_in,d_out,context_length,0.0,2)

context_vector = mha(batch)

print(context_vector)
print(f"context vector shape: {context_vector.shape}")



tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
context vector shape: torch.Size([2, 6, 4])


So now we will write a efficient implementation of the multi-head attention mechanism.
i.e. instead of multiply different causal attention heads, we will multiply all the heads at once.

for ex: 
![](efficient_mha.png)

In [32]:
class MultiHeadAttention(torch.nn.Module):
    def __init__(self,d_in,d_out,context_length,dropout,num_heads,qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads

        self.W_query = torch.nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_key = torch.nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_value = torch.nn.Linear(d_in,d_out,bias=qkv_bias)
        self.output_proj = torch.nn.Linear(d_out,d_out)
        self.dropout = torch.nn.Dropout(dropout)
        self.register_buffer('mask',torch.triu(torch.ones(context_length,context_length),diagonal=1))
        
    def forward(self,x):
       b, num_tokens, d_in = x.shape

       keys = self.W_key(x)
       queries = self.W_query(x)
       values = self.W_value(x)

       # We are splitting the d_out dimension into num_heads and head_dim
       # because we want to apply causal attention to each head separately
       # so it's going from (b,num_tokens,d_out) to (b,num_tokens,num_heads,head_dim)
       keys = keys.view(b,num_tokens,self.num_heads,self.head_dim)
       queries = queries.view(b,num_tokens,self.num_heads,self.head_dim)
       values = values.view(b,num_tokens,self.num_heads,self.head_dim)

       # now let's group them by heads and apply causal attention to each head separately
       # i.e. we are going from (b,num_tokens,num_heads,head_dim) to (b,num_heads,num_tokens,head_dim)
       keys = keys.transpose(1,2)
       queries = queries.transpose(1,2)
       values = values.transpose(1,2)

       attention_scores = queries @ keys.transpose(2,3) # dot product of queries and keys

       mask_bool = self.mask.bool()[:num_tokens,:num_tokens] 

       attention_scores.masked_fill_(mask_bool,-torch.inf)

       attention_weights = torch.softmax(
           attention_scores / keys.shape[-1]**0.5,dim=-1
       )

       attention_weights = self.dropout(attention_weights)

       context_vector = attention_weights @ values

       ## now since in output we have to concatenate all the heads along the last dimension
       ## so we need to go from (b,num_heads,num_tokens,head_dim) to (b,num_tokens,num_heads*head_dim)
       context_vector = context_vector.transpose(1,2)
       context_vector = context_vector.contiguous().view(b,num_tokens,self.d_out)

       context_vector = self.output_proj(context_vector) # projecting the context vector back to d_out dimension
       return context_vector

In [33]:
torch.manual_seed(123)

# Define the tensor with 3 rows and 6 columns
inputs = torch.tensor(
    [[0.43, 0.15, 0.89, 0.55, 0.87, 0.66],  # Row 1
     [0.57, 0.85, 0.64, 0.22, 0.58, 0.33],  # Row 2
     [0.77, 0.25, 0.10, 0.05, 0.80, 0.55]]  # Row 3
)

batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape) 

batch_size, context_length, d_in = batch.shape
d_out = 6
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

torch.Size([2, 3, 6])
tensor([[[ 0.1569, -0.0873,  0.0210,  0.0215, -0.3243, -0.2518],
         [ 0.1117, -0.0547,  0.0406, -0.0213, -0.3251, -0.2993],
         [ 0.1196, -0.0491,  0.0318, -0.0635, -0.2788, -0.2578]],

        [[ 0.1569, -0.0873,  0.0210,  0.0215, -0.3243, -0.2518],
         [ 0.1117, -0.0547,  0.0406, -0.0213, -0.3251, -0.2993],
         [ 0.1196, -0.0491,  0.0318, -0.0635, -0.2788, -0.2578]]],
       grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 3, 6])
